In [3]:
from keras.applications import ResNet50
from keras.layers import Input, BatchNormalization, Dense, Activation
from keras.models import Model

def model() :
    
    input_shape = Input(shape=(224,224,3))
    model = ResNet50(include_top=False, input_tensor=input_shape, pooling='max')

    x = model.output
    x = Dense(1024,kernel_initializer='uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dense(512, kernel_initializer='uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(model.input, x)
    
    return model


Using TensorFlow backend.


In [3]:
def set_model(categori) :
    if categori == 'smile' :
        smile_model = model()
        smile_model.load_weights("resnet_50_2class_smile_others_84%.h5")
        #smile_model.summary()
        print('Finish setting the smile_model')
        return smile_model
    
    if categori == 'angry' :
        angry_model = model()
        angry_model.load_weights("resnet_50_2class_angry_others_86%.h5")
        #angry_model.summary()
        print('Finish setting the angry_model')
        return angry_model
    
    if categori == 'sad' :
        sad_model = model()
        sad_model.load_weights("resnet_50_2class_sad_others_81%.h5")
        #sad_model.summary()
        print('Finish setting the sad_model')
        return sad_model
    
    if categori == 'normal' :
        normal_model = model()
        normal_model.load_weights("resnet_50_2class_normal_others_83%.h5")
        #normal_model.summary()
        print('Finish setting the normal_model')
        return normal_model
    

In [2]:
import numpy as np

def score_list(img, models) :
    # 0 = smile
    # 1 = sad
    # 2 = angry
    # 3 = normal
    score = []
    smile = models[0].predict(img)
    score.append(smile[0])

    sad = models[1].predict(img)
    score.append(sad[0])

    angry = models[2].predict(img)
    score.append(angry[0])

    normal = models[3].predict(img)
    score.append(normal[0])
    
    return score

In [1]:
def heightest_score(score) :
    heightest_index = score.index(max(score))
    heightest_score = max(score)

    if heightest_index == 0:
        confidence = format(100 * float(score[0]), '.4f')
        display_string = str(confidence) + '% smile'

    elif heightest_index == 1:
        confidence = format(100 * float(score[1]), '.4f')
        display_string = str(confidence) + '% sad'

    elif heightest_index == 2:
        confidence = format(100 * float(score[2]), '.4f')
        display_string = str(confidence) + '% angry'

    elif heightest_index == 3:
        confidence = format(100 * float(score[3]), '.4f')
        display_string = str(confidence) + '% normal'
        
    return display_string, heightest_index, heightest_score

In [1]:

import cv2

def extract_face(frame, size=0.5) :
    casecade_file = cv2.CascadeClassifier(r"C:\Users\Sub\Anaconda3\envs\sub\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml")
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = casecade_file.detectMultiScale(img, 1.3, 1)
    
    if faces is() :
        return frame, []
    
    for (x,y,w,h) in faces :
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 100), 1)
        
        roi = frame[y:y+h, x:x+w]
        
#         roi = cv2.equalizeHist(roi)
        roi = cv2.resize(roi, dsize=(224,224), interpolation=cv2.INTER_CUBIC)
#         roi = cv2.cvtColor(roi, cv2.COLOR_GRAY2RGB)
#         roi = cv2.equalizeHist(roi)
        roi = roi.astype('float')/255
        roi = roi.reshape((1,224,224,3))
        
        #img = frame 위에 사각형이 그려진 사진
        #roi = 관심영역
    return frame, roi